In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch
import os

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
import pandas as pd
from datasets import Dataset

data = pd.read_csv("/content/wikihowAll.csv")  

data = data.dropna(subset=["text", "headline"]).reset_index(drop=True)

train_df = data.sample(frac=0.9, random_state=42)
val_df = data.drop(train_df.index)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print(f"Training Dataset Example: {train_dataset[0]}")
print(f"Validation Dataset Example: {val_dataset[0]}")


Training Dataset Example: {'headline': '\nInsert the black probe into the common terminal and the red probe into the terminal marked for measuring ohms, volts or diode test.\n\n,\nUse the selector knob to choose the test diode function.,\nTurn the power off to the circuit.\n\n,\nTest forward bias.,\nReverse the probes to test reverse bias.,\nA reading of "OL" or 0 while testing forward bias, and 0 while testing reverse bias indicates that the diode is bad.', 'title': 'How to Use a Digital Multimeter4', 'text': ', This may be represented by a symbol representing a diode, an arrow pointing at a vertical line.\n\n,, Place the red probe on the positive side of the diode and the black probe on the negative side. If you get a reading of less than 1 but more than 0, the forward bias is good.\n\n, If the display reads "OL (overload)," this indicates that reverse bias is good.\n\n,\n\n\nSome multimeters will beep when the reading is less than 1. This doesn\'t necessarily indicate a good diode, 

In [ ]:

def tokenize_and_save_in_batches(dataset, tokenizer, filename, batch_size=1000, use_gpu=False):
    tokenized_data = []
    device = torch.device("cuda" if use_gpu and torch.cuda.is_available() else "cpu")

    for i in tqdm(range(0, len(dataset), batch_size), desc="Tokenizing Dataset in Batches"):
        batch = dataset.select(range(i, min(i + batch_size, len(dataset))))
        texts = [f"summarize: {example['text']}" for example in batch]
        summaries = [example['headline'] for example in batch]

        inputs = tokenizer(
            texts, max_length=512, truncation=True, padding="max_length", return_tensors="pt"
        ).to(device)
        targets = tokenizer(
            summaries, max_length=128, truncation=True, padding="max_length", return_tensors="pt"
        ).to(device)

        for input_ids, attention_mask, labels in zip(inputs["input_ids"], inputs["attention_mask"], targets["input_ids"]):
            tokenized_data.append({
                "input_ids": input_ids.cpu(),
                "attention_mask": attention_mask.cpu(),
                "labels": labels.cpu(),
            })

        del inputs, targets
        torch.cuda.empty_cache()

    torch.save(tokenized_data, filename)
    print(f"Tokenized dataset saved to {filename}")


# tokenize_and_save_in_batches(train_dataset, tokenizer, filename="train_tokenized.pt", batch_size=1000, use_gpu=True)
# tokenize_and_save_in_batches(val_dataset, tokenizer, filename="val_tokenized.pt", batch_size=1000, use_gpu=True)


In [ ]:
# import shutil

# output_dir = "/kaggle/working/tokenized_data"
# os.makedirs(output_dir, exist_ok=True)
# shutil.move("train_tokenized.pt", os.path.join(output_dir, "train_tokenized.pt"))
# shutil.move("val_tokenized.pt", os.path.join(output_dir, "val_tokenized.pt"))

# from IPython.display import FileLink
# print("Download Tokenized Files:")
# display(FileLink(os.path.join(output_dir, "train_tokenized.pt")))
# display(FileLink(os.path.join(output_dir, "val_tokenized.pt")))


In [ ]:
def load_tokenized_data(filename):
    tokenized_data = torch.load(filename)
    print(f"Tokenized dataset loaded from {filename}")
    return tokenized_data

train_tokenized = load_tokenized_data("/content/drive/MyDrive/Colab Notebooks/train_tokenized_t5_itr2.pt")
val_tokenized = load_tokenized_data("/content/val_tokenized_t5_itr2.pt")


<ipython-input-9-c0af4e9f44cc>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tokenized_data = torch.load(filename)


Tokenized dataset loaded from /content/drive/MyDrive/Colab Notebooks/train_tokenized_t5_itr2.pt
Tokenized dataset loaded from /content/val_tokenized_t5_itr2.pt


In [ ]:
from torch.utils.data import Dataset

class TokenizedDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        if isinstance(idx, list):
            return [self.tokenized_data[i] for i in idx]
        return self.tokenized_data[idx]


batch_size = 8
train_loader = DataLoader(TokenizedDataset(train_tokenized), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TokenizedDataset(val_tokenized), batch_size=batch_size)


In [ ]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def compute_metrics(predictions, references):
    decoded_preds = [pred.strip() for pred in predictions]
    decoded_refs = [[ref.strip()] for ref in references]

    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_refs)["bleu"]
    rouge_score = rouge.compute(predictions=decoded_preds, references=decoded_refs)

    return bleu_score, rouge_score


In [ ]:
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

def save_checkpoint(model, optimizer, epoch, checkpoint_dir):
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pt")
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        },
        checkpoint_path,
    )
    print(f"Checkpoint saved for epoch {epoch}: {checkpoint_path}")

def load_checkpoint(model, optimizer, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"] + 1 
    print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
    return start_epoch


In [ ]:
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

resume_checkpoint = '/content/checkpoints/checkpoint_epoch_3.pt'

if resume_checkpoint:
    start_epoch = load_checkpoint(model, optimizer, resume_checkpoint)
else:
    start_epoch = 1

for epoch in range(start_epoch, epochs + 1):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch}"):
        inputs = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch}: Training Loss = {avg_loss}")

    model.eval()
    predictions, references = [], []
    for batch in tqdm(val_loader, desc=f"Validating Epoch {epoch}"):
        inputs = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"]

        with torch.no_grad():
            outputs = model.generate(input_ids=inputs, attention_mask=attention_mask, max_length=128, num_beams=4, early_stopping=True)
            decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            decoded_refs = tokenizer.batch_decode(labels, skip_special_tokens=True)

        predictions.extend(decoded_preds)
        references.extend(decoded_refs)

    bleu_score, rouge_score = compute_metrics(predictions, references)
    print(f"Epoch {epoch}: BLEU Score = {bleu_score}")
    print(f"Epoch {epoch}: ROUGE Score = {rouge_score}")

    save_checkpoint(model, optimizer, epoch, checkpoint_dir)
    save_checkpoint(model, optimizer, epoch, "/content")


<ipython-input-12-d6052a79c8c4>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


Checkpoint loaded. Resuming from epoch 4


Training Epoch 4: 100%|██████████| 24109/24109 [34:53<00:00, 11.52it/s]


Epoch 4: Training Loss = 1.4679429802283837


Validating Epoch 4: 100%|██████████| 2679/2679 [1:27:35<00:00,  1.96s/it]


Epoch 4: BLEU Score = 0.10989139697963175
Epoch 4: ROUGE Score = {'rouge1': 0.3067318682610807, 'rouge2': 0.11369896492866727, 'rougeL': 0.2561906251468176, 'rougeLsum': 0.25614022391772784}
Checkpoint saved for epoch 4: ./checkpoints/checkpoint_epoch_4.pt
Checkpoint saved for epoch 4: /content/checkpoint_epoch_4.pt


Training Epoch 5: 100%|██████████| 24109/24109 [34:52<00:00, 11.52it/s]


Epoch 5: Training Loss = 1.4492261756301603


Validating Epoch 5: 100%|██████████| 2679/2679 [1:25:30<00:00,  1.92s/it]


Epoch 5: BLEU Score = 0.10961517913965366
Epoch 5: ROUGE Score = {'rouge1': 0.3133910229605922, 'rouge2': 0.11720121300750322, 'rougeL': 0.2608307409913394, 'rougeLsum': 0.26076834074535604}
Checkpoint saved for epoch 5: ./checkpoints/checkpoint_epoch_5.pt
Checkpoint saved for epoch 5: /content/checkpoint_epoch_5.pt


In [18]:
# save_checkpoint(model, optimizer, epoch, "/content/checkpoints/")

In [ ]:
def generate_summary(text, model, tokenizer, max_length=128, num_beams=4):
    model.eval()
    input_ids = tokenizer(
        f"summarize: {text}", return_tensors="pt", truncation=True, padding="max_length", max_length=512
    ).input_ids.to(device)

    outputs = model.generate(
        input_ids=input_ids, max_length=max_length, num_beams=num_beams, early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

example_texts = [
    "Artificial intelligence is the simulation of human intelligence by machines.",
    "Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from data.",
]

for i, text in enumerate(example_texts):
    summary = generate_summary(text, model, tokenizer)
    print(f"Input {i + 1}: {text}")
    print(f"Generated Summary {i + 1}: {summary}")


Input 1: Artificial intelligence is the simulation of human intelligence by machines.
Generated Summary 1: Use Artificial Intelligence.
Input 2: Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from data.
Generated Summary 2: Understand machine learning.
